In [2]:
from tqdm.auto import tqdm
import numpy as np
import os, glob, pickle
import pandas as pd
from collections import Counter
import re, string

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

import warnings
warnings.simplefilter("ignore")

[nltk_data] Downloading package stopwords to /home/hemant/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
!ls data/3a/

data.zip  sentiment_test.csv  sentiment_train.csv


# 3A

In [4]:
def remove_punctuation(text): 
    translator = str.maketrans('', '', string.punctuation) 
    return text.translate(translator) 

def pre(txt):
    dummy = []
    for i in txt.split():
        if '@' in i: dummy.append('@someuser')
        elif '#' in  i: dummy.append('#someuser')
        elif i in stopwords: continue
        else: dummy.append(remove_punctuation(i))
    
    return dummy

In [18]:
# train_3a = pd.read_csv('data/3a/sentiment_train.csv') # 11.2 lakhs
# train_3ax = [pre(i.lower()) for i in tqdm(train_3a['5'])]
# train_3ay = [i for i in train_3a['0']]

train = {
            'x': train_3a['5'],
            'y': train_3a['0']
}

# test_3a = pd.read_csv('data/3a/sentiment_test.csv') # 4.8 lakhs
# test_3ax = [pre(i.lower()) for i in tqdm(test_3a['5'])]
# test_3ay = [i for i in test_3a['0']]

test = {
            'x': test_3a['5'],
            'y': test_3a['0']
}

train_x = train_3ax + test_3ax
train_y = train_3ay + test_3ay

In [6]:
n_gram = 2

for n_gram in range(1,n_gram+1):
    tags = [i[j:j+n_gram] for i in tqdm(train_x) for j in range(len(i) - n_gram + 1)]
    print(f" Number of {n_gram} grams present in the dataset: {len(tags)/1000000} million")

    unique_data = [list(x) for x in tqdm(set(tuple(x) for x in tags))]
    print(f" Number of unique {n_gram} grams present in the dataset: {len(unique_data)/1000000} million")


 Number of 1 grams present in the dataset: 12.976944 million



 Number of unique 1 grams present in the dataset: 0.508706 million



 Number of 2 grams present in the dataset: 11.377076 million



 Number of unique 2 grams present in the dataset: 4.552656 million


#### Dummy test example

In [36]:
# train = {
#             'x': ['chinese beijing chinese', 'chinese chinese shanghai', 'chinese macao','tokyo japan chinese'],
#             'y': [1,1,1,2]
# }

# test = {
#             'x': ['chinese chinese chinese tokyo japan'],
#             'y': [1]
# }

### Naive bayes

In [117]:
class naive_bayes_():
    def __init__(self,train, test):
        n = 4
        self.train_x = [pre(i.lower()) for i in tqdm(train['x'][:n])]
        self.train_y = [i for i in tqdm(train['y'][:n])]
        
        if test:
            self.test_x = [pre(i.lower()) for i in tqdm(test['x'][:n])]
            self.test_y = [i for i in tqdm(test['y'][:n])]

            self.data_x, self.data_y = self.train_x + self.test_x, self.train_y + self.test_y
        
        else: 
            self.test_x = None
            self.data_x = self.train_x
            self.data_y = self.train_y 
        
        self.vocab = sorted([i for i in set([j for i in tqdm(self.data_x) for j in i])])
        
    def fit(self):
        
        dummy = Counter(self.train_y)
        prior = {i:dummy[i]/len(self.train_y) for i in set(dummy)}
        print(prior.keys())

        matrix = {i:np.zeros(len(self.vocab)) for i in prior.keys()}
        self.vti = {i:d for d,i in enumerate(self.vocab)}
        
        dummy = {i:len(self.vocab) for i in prior.keys()}
        for i in tqdm(range(len(self.train_y))):
            count = Counter(self.train_x[i])
            dummy[self.train_y[i]] += sum(count.values())
            
            for k,v in count.items(): 
                matrix[self.train_y[i]][self.vti[k]]+= v
        
        for k,v in matrix.items():
            matrix[k] += 1
            matrix[k]/= dummy[k]
        
        self.prior = prior
        self.matrix = matrix
        
        return prior, matrix
        
    def predict(self,test_x):
        
        pred_y = []
        label = {i:0 for i in self.prior.keys()}
        
        for i in test_x:
            for j in label.keys():
                prob = self.prior[j]
                for w in i.split():
                    prob *= self.matrix[j][self.vti[w]]
                label[j] = prob
            pred_y.append(label)
        
        return pred_y


In [118]:
naive_bayes = naive_bayes_(train,test)
prior, matrix = naive_bayes.fit()
# len(naive_bayes.vocab),len([j for i in naive_bayes.data_x for j in i]),prior

dict_keys([1, 2])


In [119]:
matrix

{1: array([0.14285714, 0.42857143, 0.07142857, 0.14285714, 0.14285714,
        0.07142857]),
 2: array([0.11111111, 0.22222222, 0.22222222, 0.11111111, 0.11111111,
        0.22222222])}

In [120]:
naive_bayes.predict(test['x'])

[{1: 0.00030121377997263036, 2: 0.00013548070246744226}]

# 3B

In [ ]:
header_list = ['id', 'tweet', 'emo', 'inten']
train = pd.read_table('data/train/out.txt', names=header_list)
train.drop('id',  axis='columns', inplace=True)
train_x = [i for i in train['tweet']]
train_y = [i for i in train['emo']]

dev = pd.read_table('data/test/out.txt', names=header_list)
dev.drop('id',  axis='columns', inplace=True)
dev_x = [i for i in dev['tweet']]
dev_y = [i for i in dev['emo']]

In [ ]:
Counter(train['emo']), len(train), Counter(dev['emo']), len(dev)

In [ ]:
!pip install vaderSentiment --user

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
analyzer = SentimentIntensityAnalyzer()
for sentence in x:
    vs = analyzer.polarity_scores(sentence)
    print("{:-<65} {}".format(sentence, str(vs)))

In [ ]:
from sklearn import preprocessing
def convert(data):
    number = preprocessing.LabelEncoder()
    return number.fit_transform(data)
    
train=convert(x)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(x,y)

In [ ]:
tree.plot_tree(clf) 